In [ ]:
import torch
from networks import *
import torch.nn as nn
from datasets import data_merge
from torch.utils.data import DataLoader
from transformers import *
from utils import *
import numpy as np
from loss import *

pth_path = './weights/loo_photo_SSAN_paper.pth'
model = get_model(4000)
model = nn.DataParallel(model)
model.load_state_dict(torch.load(pth_path)['state_dict'])
model.to('cuda')
model.eval()

data_path = './data_for_training_SSAN_form_temp'
data_bank = data_merge(data_path)
dataset = data_bank.get_datasets(train=True, loo_domain="", img_size=256, transform=transformer_train_ImageNet())
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

all_domain = {}

all_domain['photo'] = {'0': 0, '1': 1}
all_domain['replay_clear'] = {'0': 2, '1': 3}
all_domain['replay_notclear'] = {'0': 4, '1': 5}
all_domain['zalo'] = {'0': 6, '1': 7}

# all_domain['photo']['0'] = 0
# all_domain['photo']['1'] = 1
# all_domain['replay_clear']['0'] = 2
# all_domain['replay_clear']['1'] = 3
# all_domain['replay_notclear']['0'] = 4
# all_domain['replay_notclear']['1'] = 5
# all_domain['zalo']['0'] = 6
# all_domain['zalo']['1'] = 7

features = []
new_labels = []

with torch.no_grad():
    for i, sample_batched in enumerate(dataloader):
        image_x, label, UUID = sample_batched["image_x"].cuda(), sample_batched["label"], sample_batched["UUID"]
        cls_x1_x1, fea_x1_x1, fea_x1_x2, domain_invariant = model(image_x, image_x)
        features.append(fea_x1_x1.cpu().numpy().flatten())
        new_labels.append(all_domain[UUID[0]][str(int(label[0][0]))])

features = np.array(features)
new_labels = np.array(new_labels)